In [1]:
s = '0.05,0.05'
list(map(float, s.split(',')))

[0.05, 0.05]

In [1]:
import numpy as np

In [9]:
points = np.random.rand(3, 2)
edge_lengths = np.random.rand(3) * 10
points, edge_lengths

(array([[0.43080198, 0.61076721],
        [0.8872531 , 0.14902532],
        [0.36969174, 0.9123428 ]]),
 array([0.09797866, 4.98647157, 3.98737275]))

In [10]:
for p, e in zip(points, edge_lengths):
    print(p, e)

[0.43080198 0.61076721] 0.09797865613867085
[0.8872531  0.14902532] 4.986471570121418
[0.36969174 0.9123428 ] 3.9873727541394253


In [19]:
a = round(edge_lengths)
print(a)

TypeError: type numpy.ndarray doesn't define __round__ method

In [21]:
round(-1.011)

-1

In [1]:
import yaml

In [4]:
cfg_path = 'configs/default.yaml'
with open(cfg_path, 'r') as stream:
    cfg = yaml.load(stream, Loader=yaml.FullLoader)

In [5]:
model_cfgs = cfg['model_configs']
loader_kwargs = cfg['loader_kwargs']
train_cfgs = cfg['train_configs']
distributed_cfgs = cfg['distributed_configs']

In [8]:
cfg['distributed_configs']

{'distributed': False, 'world_size': 2, 'local_rank': 0}

In [7]:
distributed_cfgs['world_size'] = 2

In [1]:
import yaml
import os

In [26]:
yaml_path = './configs/default.yaml'
with open(yaml_path, 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)
config

{'model_configs': {'type': 'simple_conv',
  'kernel_size': 7,
  'depths': [1, 2, 1],
  'dims': [4, 8, 16],
  'act': 'relu',
  'norm': 'BN'},
 'dataset_configs': {'mean': 0.1094, 'std': 0.366, 'fig_resize': 64},
 'loader_kwargs': {'num_workers': 4,
  'pin_memory': True,
  'prefetch_factor': 4,
  'persistent_workers': False},
 'train_configs': {'dataset_dir': '/home/wangyh/01-Projects/03-my/Datasets/polygons_unfilled_64_3',
  'resume': False,
  'resume_path': None,
  'mode': 'train',
  'batch_size': 512,
  'seed': 1026,
  'device': 'cuda:2',
  'amp': True},
 'optim_kwargs': {'optim_type': 'AdamW',
  'lr': 0.00012,
  'momentum': 0.9,
  'weight_decay': 0.005},
 'schedule_configs': {'schedule_type': 'cosine_warm',
  'max_epoch': 90,
  'cos_T': 25,
  'cos_mul': 2,
  'cos_iters': 2},
 'distributed_configs': {'distributed': False,
  'gpu_ids': 0,
  'world_size': 1,
  'local_rank': 0,
  'port': 4250},
 'log_configs': {'log_dir': 'test_runs2',
  'save_epoch_interval': 5,
  'snapshot_interval': 1

In [27]:
log_cfgs = config['log_configs']
log_cfgs['log_dir']

'test_runs2'

In [28]:
os.makedirs(log_cfgs['log_dir'], exist_ok=True)

In [2]:
os.path.split('/home/wangyh/01-Projects/03-my/Datasets')

('/home/wangyh/01-Projects/03-my', 'Datasets')

In [3]:
from prettytable import PrettyTable

In [4]:
table = PrettyTable()

In [5]:
table.add_row(['1','server01','服务器01','172.16.0.1'])
table.add_row(['2','server02','服务器02','172.16.0.2'])
table.add_row(['3','server03','服务器03','172.16.0.3'])
table.add_row(['4','server04','服务器04','172.16.0.4'])
table.add_row(['5','server05','服务器05','172.16.0.5'])
table.add_row(['6','server06','服务器06','172.16.0.6'])
table.add_row(['7','server07','服务器07','172.16.0.7'])
table.add_row(['8','server08','服务器08','172.16.0.8'])
table.add_row(['9','server09','服务器09','172.16.0.9'])
print(table)

+---------+----------+----------+------------+
| Field 1 | Field 2  | Field 3  |  Field 4   |
+---------+----------+----------+------------+
|    1    | server01 | 服务器01 | 172.16.0.1 |
|    2    | server02 | 服务器02 | 172.16.0.2 |
|    3    | server03 | 服务器03 | 172.16.0.3 |
|    4    | server04 | 服务器04 | 172.16.0.4 |
|    5    | server05 | 服务器05 | 172.16.0.5 |
|    6    | server06 | 服务器06 | 172.16.0.6 |
|    7    | server07 | 服务器07 | 172.16.0.7 |
|    8    | server08 | 服务器08 | 172.16.0.8 |
|    9    | server09 | 服务器09 | 172.16.0.9 |
+---------+----------+----------+------------+


In [14]:
table.vertical_char

'|'